# Age prediction from eeg features

In [1]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
from utils import visualize, create_dataset_age, create_dataset_mri, cv
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import seaborn as sns
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import pandas as pd
from sklearn.linear_model import Lasso

## Create dataset

In [2]:
target = 'WISC_FSIQ'
# use average and clusters eeg features
# consider all the patients
data = create_dataset_mri(SCORE = target)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
data = data.rename(columns={'id':'ID'})

print(data.shape)
data.dropna(axis=0, inplace=True, subset=[target])
print(data.shape)

/Users/alestolfo/anaconda2/envs/ds-lab/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (50,78,80,91,92,93,94,95,105,106,107,108,109,119,120,121,123,133,134,135,137,276,291,292,295,296,297,300,301,302,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(1146, 371)
(842, 371)


In [3]:
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

## Fix a baseline (mean age)

In [4]:
# Baseline
mean = np.mean(y_train)
print('mean_age = {}'.format(mean))
baseline_MSE = sum((mean - y_train)**2)/len(y_train)
print('baseline_MSE = {}'.format(baseline_MSE))

mean_age = 97.56724367509986
baseline_MSE = 286.4132545864273


In [5]:
print('Baseline test error: {}'.format(sum((mean - y_test)**2)/len(y_test)))

Baseline test error: 264.591413182554


# Explore different regressors and feature selection procedures

## Use AVERAGE features

## ExtraTrees + SVR

In [6]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median'],
            'regression__C' : [0.5, 0.8, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
            'feat_select__threshold' : [-np.inf],
            'feat_select__max_features' : [10, 20, 50, 60]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring =  'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test R^2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.1min finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.2min finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


...

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.8min finished



Average expected test MSE: 268.84765232219144

Average expected test R^2: 0.05391268031476135

Average expected test mae: 12.997317871984142
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 12.6min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'feat_select__max_features': 60, 'feat_select__threshold': -inf, 'imputation__strategy': 'mean', 'regression__C': 2, 'regression__epsilon': 0.1, 'regression__kernel': 'rbf'}
True test error: 245.77328576600001
Test r2: 0.06980816191485228
Test mae: 12.713763955484815


# XGBoost

In [8]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 4, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring =  'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen   
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.0min finished


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.0min finished


Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.3min finished



Average expected test MSE: 291.97877580816186

Average expected test r2: -0.02908417031466335

Average expected test mae: 13.64394303257418
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.8min finished


Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__booster__alpha': 0, 'regression__booster__max_depth': 3}
True test error: 297.06027861573773
Test r2: -0.12430057533081729
Test mae: 14.183465643243474


## Lasso

In [9]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('regression', Lasso())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__alpha' : [1, 1.2, 1.4, 1.7, 1.9, 2, 2.3, 2.5, 3, 3.5, 4, 6],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = 'neg_mean_squared_error', cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2, mae) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2=True, want_mae = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
print('\nAverage expected test mae: {}'.format(np.mean(mae)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))
print('Test mae: {}'.format(mean_absolute_error(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


...

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



Average expected test MSE: 270.8750347544645

Average expected test r2: 0.04317580313070708

Average expected test mae: 13.080682421338517
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Selected hyperparameters: {'imputation__strategy': 'mean', 'regression__alpha': 1.2}
True test error: 255.9662798921394
Test r2: 0.031230169549921416
Test mae: 12.977698085911452


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.6min finished
